In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn
import pandas as pd
%matplotlib inline

In [2]:
ratings = pd.read_csv('dataSets/jester_dataset_2/jester_ratings.dat', sep='\t\t' , names=['user', 'item', 'rating'])
ratings.head()

/Users/i506000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,user,item,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875


In [3]:
ratings.rating += 10
print(ratings)

          user  item  rating
0            1     5  10.219
1            1     7   0.719
2            1     8   0.719
3            1    13   3.219
4            1    15  10.875
5            1    16   0.344
6            1    17   0.969
7            1    18   2.531
8            1    19   1.281
9            1    20   0.844
10           1    21   2.812
11           1    22   1.219
12           1    23   1.469
13           1    24   2.094
14           1    25   2.531
15           1    89  19.812
16           1    50  19.906
17           1   102  10.750
18           1   103   5.000
19           1   104  12.938
20           1   105  12.000
21           1   106   9.844
22           1   107  12.031
23           1   108  15.688
24           1   109  19.656
25           1    87  18.000
26           1    93  19.312
27           1    76  19.312
28           1    65  18.781
29           1    72  18.781
...        ...   ...     ...
1761409  63978    81  17.531
1761410  63978   142  17.844
1761411  63978

In [4]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [5]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 10)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [6]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))

In [7]:
all_recs = pd.concat(all_recs, ignore_index=True)
print(all_recs)

        item      score   user  rank Algorithm
0         89  18.857313      9     1  ItemItem
1         53  18.668875      9     2  ItemItem
2        105  18.640842      9     3  ItemItem
3         35  18.333825      9     4  ItemItem
4        132  18.304901      9     5  ItemItem
5        129  18.177356      9     6  ItemItem
6         19  18.152216      9     7  ItemItem
7         50  18.139066      9     8  ItemItem
8         32  18.091278      9     9  ItemItem
9         72  18.043625      9    10  ItemItem
10       129  14.516198     10     1  ItemItem
11       126  14.467068     10     2  ItemItem
12       127  14.393447     10     3  ItemItem
13        72  14.324526     10     4  ItemItem
14       148  14.052576     10     5  ItemItem
15       138  13.993088     10     6  ItemItem
16       114  13.935749     10     7  ItemItem
17       150  13.814655     10     8  ItemItem
18       111  13.738967     10     9  ItemItem
19       134  13.564794     10    10  ItemItem
20       143 

In [8]:
test_data = pd.concat(test_data, ignore_index=True)

In [9]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
print(results)

/Users/i506000/anaconda3/lib/python3.7/site-packages/lenskit/metrics/topn.py:104: RuntimeWarning: invalid value encountered in double_scalars
  return achieved / ideal


                     ndcg
user  Algorithm          
1     ItemItem   0.149753
2     ItemItem   0.220130
3     ItemItem   0.000000
4     ItemItem   0.000000
5     ItemItem   0.330094
6     ItemItem   0.029664
7     ItemItem   0.230071
8     ItemItem   0.213642
9     ItemItem   0.004450
10    ItemItem   0.063549
11    ItemItem   0.000000
12    ItemItem   0.000000
13    ItemItem   0.349635
14    ItemItem   0.000000
15    ItemItem   0.000000
16    ItemItem   0.000000
17    ItemItem   0.250200
18    ItemItem   0.239076
19    ItemItem   0.000000
22    ItemItem   0.219192
23    ItemItem   0.371997
24    ItemItem   0.000000
25    ItemItem   0.278392
26    ItemItem   0.018992
27    ItemItem   0.000000
28    ItemItem   0.318071
29    ItemItem   0.144816
33    ItemItem   0.417957
35    ItemItem   0.000000
37    ItemItem   0.475358
...                   ...
63940 ItemItem   0.369103
63941 ItemItem   0.000000
63943 ItemItem   0.000000
63944 ItemItem   0.413998
63945 ItemItem   0.000000
63946 ItemIt

In [10]:
results.groupby('Algorithm').ndcg.mean()

Algorithm
ItemItem    0.185229
Name: ndcg, dtype: float64

In [13]:
with open("Output.txt", "w") as text_file:
    print(f"Purchase Amount: {results}", file=text_file)


In [ ]:
predict_for_user(user, items, ratings=None)